<a href="https://www.kaggle.com/code/carloscll/cll-baseline-abalone-s4e4?scriptVersionId=170844130" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Notes

- Plan 1: *None*
- Evaluation: 0.16656
- Sample Size: 5000
- How well: Fail
- LB: None
___
- Plan 2: *None*
- Evaluation:  0.17426 
- Sample Size: 100%
- How well: Good
- LB: None

# IDLE

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.metrics import mean_squared_log_error



URL_TRAIN = '/kaggle/input/playground-series-s4e4/train.csv'
URL_SAMPLE = '/kaggle/input/playground-series-s4e4/sample_submission.csv'
URL_TEST = '/kaggle/input/playground-series-s4e4/test.csv'

N_SAMPLE = 9000
TARGET = 'Rings'
SEED = 101

In [2]:
# 0. seed
np.random.seed(SEED)

# 1. load data
train = pd.read_csv(URL_TRAIN, index_col='id') 
train = train.sample(N_SAMPLE)

# 2. split data
X, y = train.drop(TARGET, axis=1), train[TARGET]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# 3. dic. models
dtr = DecisionTreeRegressor()

# 4. create pipeline
numeric_features = X.select_dtypes(exclude=['object']).columns
numeric_transformer = Pipeline(
    steps=[("scaler", StandardScaler())]) 

categorical_features = X.select_dtypes(include=['object']).columns
categorical_transformer = Pipeline(
    steps=[("encoder", OneHotEncoder(drop='if_binary'))]) 
    
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

clf = Pipeline(steps=[
     ("preprocessor", preprocessor),
                       ("classifier", dtr)])


# 5. train
clf.fit(X_train, y_train);

# Display

## Evaluation

In [3]:
y_pred_val = clf.predict(X_val)
rmsle_val = np.sqrt(mean_squared_log_error(y_val, y_pred_val))

print(f"Mean squared logarithmic error regression loss: {rmsle_val:.5f}")

Mean squared logarithmic error regression loss: 0.21227


# Submission

In [4]:
# 1. load sample and test
submission = pd.read_csv(URL_SAMPLE)
test = pd.read_csv(URL_TEST, index_col='id')

# 2. predict and assign
submission['Rings'] = clf.predict(test)

# 3. save to csv
submission.to_csv('base__dtr__public.csv', index=False)